In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
!pip install backtrader
import backtrader as bt
import pytz
from datetime import datetime, time as dtime
from datetime import time
import time as time_module
from datetime import time
from datetime import datetime, time as dtime
import time as time_mod
!pip install alpaca-py
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

#api_key="PKB9SXQSORHZSB1VIXY0"
#secret_key="q4n01bamgGErL7p261A7mu5UvHZOOTO2i6rSsXgJ"
trading_client = TradingClient('PKB9SXQSORHZSB1VIXY0', 'q4n01bamgGErL7p261A7mu5UvHZOOTO2i6rSsXgJ', paper=True)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.0/122.0 kB 4.4 MB/s eta 0:00:00


In [3]:
#Code to palce orders
# preparing orders
market_order_data = MarketOrderRequest(
                    symbol="ATNF",
                    qty=1,
                    side=OrderSide.SELL,
                    time_in_force=TimeInForce.GTC
                    )

# Market order
market_order = trading_client.submit_order(
                order_data=market_order_data
               )

In [4]:
# pip install alpaca-py
# ── Configure: API keys in env or paste below. Use paper=True for paper trading.

import os
import asyncio
from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.live import StockDataStream
from alpaca.trading.models import Position

# ======== CONFIG ========
API_KEY = os.getenv("ALPACA_API_KEY") or "YOUR_PAPER_API_KEY"
API_SECRET = os.getenv("ALPACA_API_SECRET") or "YOUR_PAPER_SECRET"
PAPER = True                           # keep True for safety
SYMBOL = "ATNF"                        # change to your symbol
QTY = 1                                # shares per trade
PRICE_FLOOR = 5.00                     # only trade if price > $5
COOLDOWN_SEC = 10                      # avoid rapid re-orders
SKIP_FIRST_MIN = 1                     # skip first minute after open
SKIP_LAST_MIN = 1                      # skip last minute before close
# ========================

trading = TradingClient(API_KEY, API_SECRET, paper=PAPER)

last_order_time = None
in_long = False                        # simple local state; also checked against broker
et = ZoneInfo("America/New_York")

def now_et():
    return datetime.now(tz=et)

def market_open_now() -> bool:
    try:
        clk = trading.get_clock()
        if not clk.is_open:
            return False
        cal = trading.get_calendar(from_=now_et().date(), to=now_et().date())[0]
        open_dt = cal.open.replace(tzinfo=et)
        close_dt = cal.close.replace(tzinfo=et)
        t = now_et()
        if t < open_dt + timedelta(minutes=SKIP_FIRST_MIN):
            return False
        if t > close_dt - timedelta(minutes=SKIP_LAST_MIN):
            return False
        return True
    except Exception:
        return False

def broker_thinks_long(symbol: str) -> bool:
    try:
        pos: Position = trading.get_open_position(symbol)
        return float(pos.qty) > 0
    except Exception:
        return False

def have_open_order(symbol: str) -> bool:
    try:
        open_orders = trading.get_orders(status="open", symbols=[symbol])
        return len(open_orders) > 0
    except Exception:
        return False

def cooldown_ok() -> bool:
    global last_order_time
    if last_order_time is None:
        return True
    return (datetime.now(timezone.utc) - last_order_time).total_seconds() >= COOLDOWN_SEC

def guardrails(price: float) -> tuple[bool, str]:
    if not market_open_now():
        return False, "market_closed_or_buffer"
    if price is None or price <= 0:
        return False, "bad_price"
    if price <= PRICE_FLOOR:
        return False, "price_below_or_equal_5"
    acct = trading.get_account()
    if acct.trading_blocked:
        return False, "trading_blocked"
    if not cooldown_ok():
        return False, "cooldown_active"
    if have_open_order(SYMBOL):
        return False, "pending_order_exists"
    return True, ""

def submit_order(side: OrderSide, qty: int):
    global last_order_time
    req = MarketOrderRequest(
        symbol=SYMBOL,
        qty=qty,
        side=side,
        time_in_force=TimeInForce.DAY,
    )
    o = trading.submit_order(req)
    last_order_time = datetime.now(timezone.utc)
    print(f"[{now_et().strftime('%H:%M:%S')}] ORDER {o.side} {o.symbol} x{o.qty} id={o.id}")
    return o

async def main():
    global in_long
    # Sync local state with broker at start
    in_long = broker_thinks_long(SYMBOL)

    stream = StockDataStream(API_KEY, API_SECRET, paper=PAPER)

    @stream.on_trade(SYMBOL)
    async def on_trade(trade):
        global in_long
        price = float(trade.price)
        ts = trade.timestamp.astimezone(et).strftime("%H:%M:%S")

        # --- Enforce the $5 rule before any strategy logic ---
        if price <= PRICE_FLOOR:
            # If holding and price fell to/below floor, flatten
            if in_long and not have_open_order(SYMBOL) and market_open_now():
                try:
                    submit_order(OrderSide.SELL, QTY)
                    in_long = False
                    print(f"[{ts}] EXIT (price <= {PRICE_FLOOR:.2f}) at {price:.2f}")
                except Exception as e:
                    print(f"[{ts}] SELL error: {e}")
            else:
                print(f"[{ts}] No trade: price {price:.2f} ≤ {PRICE_FLOOR:.2f}")
            return

        # Past this point, price > 5 and market session ok
        ok, why = guardrails(price)
        if not ok:
            print(f"[{ts}] Guard skip: {why} @ {price:.2f}")
            return

        # ===== Example micro-strategy =====
        # If not in a long position and price > $5, buy 1 share.
        # Replace this block with your own signal logic as needed.
        if not in_long:
            try:
                submit_order(OrderSide.BUY, QTY)
                in_long = True
                print(f"[{ts}] ENTER long at {price:.2f}")
            except Exception as e:
                print(f"[{ts}] BUY error: {e}")
        else:
            # Already long and price > 5 — do nothing (or add your scaling logic)
            print(f"[{ts}] Holding long; last={price:.2f}")

    print(f"Starting stream for {SYMBOL} (paper={PAPER}) with PRICE_FLOOR=${PRICE_FLOOR:.2f}")
    await stream.run()

if __name__ == "__main__":
    try:
        asyncio.run(main())
    except KeyboardInterrupt:
        print("Stopped.")


RuntimeError: asyncio.run() cannot be called from a running event loop

In [24]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from alpaca.trading.client import TradingClient

API_KEY = "PKB9SXQSORHZSB1VIXY0"
API_SECRET = "q4n01bamgGErL7p261A7mu5UvHZOOTO2i6rSsXgJ"
PAPER = True

trading = TradingClient(API_KEY, API_SECRET, paper=PAPER)
et = ZoneInfo("America/New_York")

# Today
today = datetime.now(et).date()

# Get today’s session
today_session = trading.get_calendar(today)[0]

# Get previous trading day (yesterday or earlier)
yesterday = today - timedelta(days=1)
prev_session = None
while not prev_session:
    cal = trading.get_calendar(yesterday)
    if cal:
        prev_session = cal[0]
    else:
        yesterday -= timedelta(days=1)

print("Today:", today_session.date, today_session.open, today_session.close)
print("Prev:", prev_session.date, prev_session.open, prev_session.close)


AttributeError: 'datetime.date' object has no attribute 'to_request_fields'

In [ ]:
py